In [11]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


Step: Load data

In [28]:
import csv
import pandas as pd
import numpy as np
import time
import os
from pyprojroot import here
from skimpy import clean_columns
from data_cleaning.fun_hot_encode_limit import fun_hot_encode_limit
import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.metrics import median_absolute_error
from math import sqrt

path_data = here("./data")
os.chdir(path_data)

In [13]:
passing_data_points = pd.read_parquet("passing_data_points.parquet")
rushing_data_points = pd.read_parquet("rushing_data_points.parquet")
receiving_data_points = pd.read_parquet("receiving_data_points.parquet")

In [14]:
passing_data_points.columns

Index(['player', 'tm', 'age', 'pos', 'g', 'gs', 'q_brec', 'cmp', 'att',
       'cmp_%', 'yds', 'td', 'td_%', 'int', 'int_%', '1_d', 'lng', 'y_a',
       'ay_a', 'y_c', 'y_g', 'rate', 'qbr', 'sk', 'sk_%', 'ny_a', 'any_a',
       '4_qc', 'gwd', 'year', 'td_points', 'yds_points', 'int_points'],
      dtype='object')

In [21]:
data_forecasting_passing = passing_data_points.rename(
    columns={"yds_points": "y", "year": "ds"}
)[["y", "ds"]]

In [36]:
data_forecasting_passing["ds"] = pd.to_datetime(
    data_forecasting_passing["ds"], format="%Y"
)

In [38]:
# Split the dataset into training and test sets
train_size = int(len(data_forecasting_passing) * 0.8)
train_df = data_forecasting_passing.iloc[:train_size]
test_df = data_forecasting_passing.iloc[train_size:]

# Fit the Prophet model on the training data
model = Prophet()
model.fit(train_df)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1


In [40]:
test_df

,y,ds
1028,1.28,2020-01-01
1029,1.16,2020-01-01
1030,1.12,2020-01-01
1031,1.12,2020-01-01
1032,1.04,2020-01-01
...,...,...
1280,0.00,2022-01-01
1281,0.00,2022-01-01
1282,0.00,2022-01-01
1283,0.00,2022-01-01


In [39]:
future = model.make_future_dataframe(
    periods=len(test_df), freq="Y", include_history=True
)

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 2277-12-31 00:00:00

In [34]:
test_df

,y,ds
1028,1.28,2020
1029,1.16,2020
1030,1.12,2020
1031,1.12,2020
1032,1.04,2020
...,...,...
1280,0.00,2022
1281,0.00,2022
1282,0.00,2022
1283,0.00,2022


In [33]:
future

,ds
0,1970-01-01 00:00:00.000002010
1,1970-01-01 00:00:00.000002011
2,1970-01-01 00:00:00.000002012
3,1970-01-01 00:00:00.000002013
4,1970-01-01 00:00:00.000002014
...,...
263,2222-12-31 00:00:00.000002020
264,2223-12-31 00:00:00.000002020
265,2224-12-31 00:00:00.000002020
266,2225-12-31 00:00:00.000002020


In [30]:


# Make predictions on the test data
forecast = model.predict(future)

# Evaluate the model using the Root Mean Squared Error (RMSE)
y_true = test_df['y'].values
y_pred = forecast['yhat'].values
rmse = sqrt(median_absolute_error(y_true, y_pred))
print(f"RMSE: {rmse}")

# Plot the forecast
fig1 = model.plot(forecast)


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1


ValueError: Found input variables with inconsistent numbers of samples: [257, 268]